In [1]:
import transformers

In [2]:
import json

with open('en_data.json') as f:
    en_data = json.load(f)

### Состав датасета:

1. Переведенные данные alpaca (в размере 6639 предложений)
2. Данные https://huggingface.co/datasets/Den4ikAI/russian_instructions_2 (30000 предложений)

Всего -- 36639 предложений.

In [3]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru")
translator("instruction")[0]['translation_text']

'инструкция'

#### Пример, как переводим данные:

In [4]:
ru_data_example = []
for i in range(5):
    temp = {}
    for k, v in en_data[i].items():
        if v != "":
            v = translator(v)[0]['translation_text']
        temp[k] = v    
    ru_data_example.append(temp)
ru_data_example

[{'instruction': 'Назовите пять средиземноморских стран.',
  'input': '',
  'output': 'Греция, Испания, Италия, Турция, Франция'},
 {'instruction': 'Нажмите на правильный параметр.',
  'input': 'Какой из следующих продуктов питания содержит больше всего сахара? A. Apple B. Orange C. Шоколад',
  'output': 'С. Шоколад'},
 {'instruction': 'Определить количество недель в году.',
  'input': '',
  'output': 'Количество недель в году составляет 52 недели.'},
 {'instruction': 'Разработка перечня из пяти шагов для обеспечения успеха проекта.',
  'input': '',
  'output': '2. Разработка подробного плана и графика; 3. создание группы специально подготовленных специалистов; 4. выявление рисков и разработка стратегий их смягчения; 5. контроль за прогрессом и своевременное внесение коррективов в учебные курсы.'},
 {'instruction': 'Напиши несколько предложений о важности воды.',
  'input': '',
  'output': 'Вода является одним из важнейших веществ на Земле, поскольку она необходима всем живым организма

#### Столько примеров всего:

In [5]:
with open('ru_data3.json') as f:
    ru_data = json.load(f)
len(ru_data)

36639

#### Как выглядят данные

In [6]:
print('Alpaca')
print(ru_data[6])
print(ru_data[10])
print()
print('HuggingFace Dataset')
print(ru_data[10000])

Alpaca
{'instruction': 'Объясните, почему следующая доля эквивалентна 1/4', 'input': '4/16', 'output': 'Доля 4/16 эквивалентна 1/4, поскольку как числители, так и знаменатели делятся на 4.'}
{'instruction': 'Как умер Юлий Цезарь?', 'input': '', 'output': 'Юлий Цезарь был убит группой в составе до 60 заговорщиков, возглавляемой Гаюсом Кассиусом Лонгинусом и Марком Джуниусом Брутусом, в Сенате на Иде (15 марта) 44 года до н. э.'}

HuggingFace Dataset
{'instruction': ' Я слышал, что иглоукалывание может облегчить боль. Это правда?', 'input': '', 'output': ' Да, иглоукалывание доказало свою эффективность в лечении многих видов боли. Он работает, стимулируя естественную систему обезболивания организма и уменьшая воспаление. Исследования показали, что иглоукалывание может помочь уменьшить такие симптомы, как головная боль, боль в пояснице, боль при артрите и даже боль при фибромиалгии.'}


In [7]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

# import utils

In [8]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведена инструкция, которая описывает задачу в сочетании с вводными данными, предоставляющими дополнительный контекст. "
        "Напишите ответ, который соответствующим образом завершает запрос.\n\n"
        "### Инструкция:\n{instruction}\n\n### Вводные данные:\n{input}\n\n### Ответ:"
    ),
    "prompt_no_input": (
        "Ниже приведена инструкция, описывающая задачу."
        "Напишите ответ, который соответствующим образом завершает запрос.\n\n"
        "### Инструкция:\n{instruction}\n\n### Ответ:"
    ),
}

In [9]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            #padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [10]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [11]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [12]:
model_name = "facebook/opt-350m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
    )

In [13]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    'facebook/opt-350m',
    model_max_length=500,
    padding_side="right",
    use_fast=False,
)

In [14]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="ru_data3.json")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [15]:
#!pip install -i https://test.pypi.org/simple/ bitsandbytes

In [16]:
training_args = transformers.TrainingArguments(learning_rate=1e-5, 
                 num_train_epochs=8,
                 per_device_train_batch_size=16,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='steps',
                 save_steps = 6000,
                 logging_steps=1000,
                 fp16=True,
                 output_dir="instruct_ft",
                 dataloader_num_workers=4)

In [18]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names


decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
 )

In [19]:
trainer = Trainer(model=model, 
                 tokenizer=tokenizer, 
                 args=training_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None, 
                 data_collator=data_collator, 
                 optimizers=(adam_bnb_optim, None),
)

/home/jovyan/.imgenv-train-ru-bert-0/lib/python3.7/site-packages/torch/utils/tensorboard/__init__.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  tensorboard.__version__


In [20]:
trainer.train()

Step,Training Loss
1000,1.175400
2000,1.079700
3000,0.968700
4000,0.917000
5000,0.861800
6000,0.811500
7000,0.779100
8000,0.718000
9000,0.701800
10000,0.643000


TrainOutput(global_step=18320, training_loss=0.6966662261163303, metrics={'train_runtime': 10504.603, 'train_samples_per_second': 27.903, 'train_steps_per_second': 1.744, 'total_flos': 2.66753568079872e+17, 'train_loss': 0.6966662261163303, 'epoch': 8.0})

In [176]:
trainer.save_model('ft_02')

In [177]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [178]:
MODEL_NAME = 'ft_02'

In [179]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=500)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [180]:
def predict_for_instruction(instruction, model):
    #text = text.replace('\n', ' ')
    prompt = ("Ниже приведена инструкция, описывающая задачу."
              "Напишите ответ, который соответствующим образом завершает запрос.\n\n"
               f"### Инструкция:\n{instruction}\n\n### Ответ:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        max_new_tokens=200,
        top_k=10, 
        top_p=0.85, 
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [181]:
instruction = "Назови самый популярный язык."
predict_for_instruction(instruction, model)

'Самый популярный язык — культурный язык.'

In [182]:
instruction = "Назови столицу США."
predict_for_instruction(instruction, model)

'Столица США - Сан-Франциско.'

Врет :(

In [183]:
def predict_for_instruction(instruction, model):
    prompt = ("Ниже приведена инструкция, описывающая задачу."
              "Напишите ответ, который соответствующим образом завершает запрос.\n\n"
               f"### Инструкция:\n{instruction}\n\n### Ответ:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        max_new_tokens=200,
        num_beams=5,
        temperature=0.4,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

### С такими параметрами отвечает относительно нормально

In [184]:
instruction = "Как можно уменьшить загрязнение воздуха?"
predict_for_instruction(instruction, model)

'Уменьшение загрязнения воздуха является возобновляемым источником энергии, таким, как солнечная энергия, энергия ветра и гидроэлектроэнергия.'

In [185]:
instruction = "Как найти работу?"
predict_for_instruction(instruction, model)

'Начните с изучения типов вакансий, доступных в интересующей вас области, а затем создайте список потенциальных работодателей, которые могут быть доступны. Вы также можете связатьс'

In [186]:
instruction = "Как найти жену?"
predict_for_instruction(instruction, model)

'Лучший способ найти жену — это попросить друзей и родственников порекомендовать их.'

In [187]:
instruction = "Что лучше: бег или плавание?"
predict_for_instruction(instruction, model)

'Бег также лучше, чем плавание.'

In [188]:
instruction = "Дай совет, как улучшить память."
predict_for_instruction(instruction, model)

'1. Регулярно практикуйтесь и сосредоточьтесь на своем дыхании. 2. Избегайте кофеина в конце дня. 3. Регулярно делайте физические упражнения, так как физическая активность может помо�'

In [189]:
instruction = "Как оставаться здоровым?"
predict_for_instruction(instruction, model)

'Одним из оставшихся здоровых привычек является сбалансированное питание, включающее большое количество фруктов и овощей, цельнозерновые продукты, нежирные белки и полезные жиры.'

In [190]:
instruction = "Как бросить курить?"
predict_for_instruction(instruction, model)

'Лучший способ бросить курить - это сосредоточиться на своем дыхании и убедиться, что у вас есть все необходимые меры предосторожности, такие, как упражнения на глубокое дыхание, прогресс'

In [191]:
instruction = "Безопасно ли есть сырые яйца?"
predict_for_instruction(instruction, model)

'Обычно считается безопасным употребление сырых яиц для большинства людей.'

In [192]:
instruction = "Чему равно 2+2?"
predict_for_instruction(instruction, model)

'Равно 2+2.'

Логично.

In [193]:
def predict_for_instruction(instruction, model):
    prompt = ("Ниже приведена инструкция, описывающая задачу."
              "Напишите ответ, который соответствующим образом завершает запрос.\n\n"
               f"### Инструкция:\n{instruction}\n\n### Ответ:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        max_new_tokens=200,
        no_repeat_ngram_size=5, 
        top_k=0, 
        top_p=0.92, 
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [194]:
instruction = "Как улучшить выносливость?"
predict_for_instruction(instruction, model)

'Чтобы улулуцировать ваши выновленные стандарты, начните с предварительного пространства для выполнения занятий, концентрации и программы. Убедитесь, что вы пребываете под наградой или прод'

### Вывод: модель что-то отвечает, но не очень хорошо